# RPS Bots

#### Game Setup

In [1]:
moves = ['R', 'P', 'S']

def winner(hero, villain):
    
    if hero == villain:
        return 'D'
    elif (hero == 'R' and villain == 'S') or \
         (hero == 'S' and villain == 'P') or \
         (hero == 'P' and villain == 'R'):
        return 'W'
    else:
        return 'L'

Track Bot History

In [2]:
hero_hist1, villain_hist1 = [], []
hero_hist2, villain_hist2 = [], []
hero_hist3, villain_hist3 = [], []
hero_hist4, villain_hist4 = [], []
hero_hist5, villain_hist5 = [], []

### Bot 1: Random

Bot 1 plays randomly and is therefore unexploitable.

In [3]:
import random

def bot1():
    
    rng = random.randint(1,3)
    
    if rng == 1:
        return 'R'
    elif rng == 2:
        return 'P'
    else:
        return 'S'

### Bot 2: Frequency Analysis

Bot 2 simply tracks what move their opponent plays most.

In [4]:
from collections import Counter

def bot2(history):
    # If no history we play a random move
    if not history:
        return bot1()
    
    count = Counter(history)
    most_common = count.most_common(1)[0][0]
    
    if most_common == 'R':
        return 'P'
    elif most_common == 'P':
        return 'S'
    else:
        return 'R'

### Bot 3: Markov Chain Model

Bot 3 predicts what their next move is based on their previous move.

In [5]:
from collections import defaultdict

markov = defaultdict(list)

def update_markov(history):
    
    if len(history) >= 2:
        markov[history[-2]].append(history[-1])

def counter(move):
    return {'R' :'P', 'P' : 'S', 'S' : 'R'}[move]
        
def bot3():
    
    update_markov(villain_hist3)

    # If no history we play a random move
    if not villain_hist3:
        return bot1()
    
    last_move = villain_hist3[-1]
    next_moves = markov[last_move]
    
    if not next_moves:
        return bot1()
    
    predicted = Counter(next_moves).most_common(1)[0][0]
    return counter(predicted)
    

### Bot 4: Trigram Model

Bot 4 predicts what their next move is based on their last 3 moves

In [6]:
ngrams = defaultdict(list)

def update_ngrams(history):
    
    if len(history) >= 4:
        key = tuple(history[-4:-1])  
        next_move = history[-1]     
        ngrams[key].append(next_move)
        
def bot4():
    update_ngrams(villain_hist4)
    
    if len(villain_hist4) < 3:
        return bot1()
    
    key = tuple(villain_hist4[-3:])
    possible_next = ngrams.get(key, [])
    
    if not possible_next:
        return bot1()
    
    predicted = Counter(possible_next).most_common(1)[0][0]
    
    return counter(predicted)    

### Bot 5: Conditional Reaction 

Bot 5 tracks both its own moves and the opponent's then predicts their next move based on both of their previous two moves.

In [7]:
reaction_map = defaultdict(list)

def update_reactions():
    if len(hero_hist5) >= 3 and len(villain_hist5) >= 3:
        prev_round_1 = (hero_hist5[-3], villain_hist5[-3])
        prev_round_2 = (hero_hist5[-2], villain_hist5[-2])
        key = (prev_round_1, prev_round_2)
        next_move = villain_hist5[-1]
        reaction_map[key].append(next_move)
        
def bot5():
    
    update_reactions
    
    if len(hero_hist5) < 2 or len(villain_hist5) < 2:
        move = bot1()
        hero_hist5.append(move)
        return move
    
    prev_round1 = (hero_hist5[-2], villain_hist5[-2])
    prev_round2 = (hero_hist5[-1], villain_hist5[-1])
    key = (prev_round1, prev_round2)
    next_moves = reaction_map.get(key, [])
    
    if not next_moves:
        move = bot1()
        hero_hist5.append(move)
        return move
    
    predicted = Counter(next_moves).most_common(1)[0][0]
    move = counter(predicted)
    hero_hist5.append(move)
    return move